# DATASCI W261: Machine Learning at Scale
## Assignment Week 5
Miki Seltzer (miki.seltzer@berkeley.edu)<br>
W261-2, Spring 2016<br>
Submission: 

## HW5.0:
### What is a data warehouse?
A data warehouse is a repository for one or multiple data sources. Data warehouses can contain relational databases.

### What is a star schema?
A star schema relates multiple fact and dimension tables, and is similar to the snowflake schema. In both schemas, fact tables are referenced by dimension tables (one or multiple). However, star schemas are denormalized, whereas snowflake schemas are normalized.

### When is it used?
A star schema is used to organize the meta data of a relational database, such as which tables can be joined, and the keys on which they can be joined.

## HW5.1:
### In the database world, what is 3NF?
3NF is shorthand for third normal form. A table is in third normal form if the following conditions hold:
- The table is already in second normal form
- Non-prime attributes of the table are non-transitively dependent on every key in the table

### Does machine learning use data in 3NF?
ML can, but does not always use data in 3NF.

### If so, why?
3NF can save a significant amount of disk space because data duplication is avoided. Additionally, if data is denormalized, then fields in the data set might be related to each other and create dependencies. This may be problematic if we are using algorithms that require independent features.

### In what form does ML consume data?
Typically, ML requires all data to be fed into an algorithm to be collected into a single source. Thus, the easiest way for ML to ingest data is for it to be denormalized.

### Why would one use log files that are denormalized?
If one needs to perform real-time analysis on log files, it may be too time consuming to join normalized log files with other tables. If log files are denormalized, they may not need any further processing (joins) to be fed into other steps of a pipeline.

## HW 5.2: Using MRJob, implement a hashside join (memory-backed map-side) for left, right and inner joins. Run your code on the  data used in HW 4.4.

### Justify which table you chose as left table in this hashside join
The two tables used were:
- **anonymous-msweb-preprocess.data:** The log file of visitors and each page that they visited (processed rows prefixed by 'C' or 'V')
- **attributes.csv:** The page ID, page name and URL of each page (prefixed by 'A' in the original data)

The attributes.csv file was very small (only 294 lines), so this is the file that I chose to store in memory. This became my **right** table.

The log file was much larger, so I streamed through this file, and used it as the **left** table.

In [2]:
# We will need these so we can reload modules as we modify them
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
%%writefile mapSideJoin.py
from mrjob.job import MRJob
from mrjob.step import MRStep
 
class join(MRJob):
    
    # Specify some custom options so we only have to write one MRJob class for each join
    def configure_options(self):
        super(join, self).configure_options()
        self.add_passthrough_option('--joinType', default='inner', )
    
    # Store attributes.csv into memory
    #  - account for multiple occurrences of keys
    #  - self.pages is dict with a list of [pageName, pageURL] pairs
    # Set joinType variable
    def mapper_init(self):
        self.pages = {}
        with open('attributes.csv','r') as myfile:
            for line in myfile:
                fields = line.strip().split(',')
                if fields[0] not in self.pages:
                    self.pages[fields[0]]=[]
                self.pages[fields[0]].append([fields[1], fields[2]])
        self.joinType = self.options.joinType
        self.seenRight = set()
    
    # RIGHT table is stored in memory (self.pages)
    # LEFT table is streamed
    # We need so keep track of which RIGHT keys we have seen
    def mapper(self, _, line):
        fields = line.split(',')
        key = fields[0]
        self.seenRight.add(key)
        if key in self.pages:
            for i in self.pages[key]:
                yield key, (fields[1], fields[2], i[0], i[1])
        elif self.joinType == 'left':
            yield key, (fields[1], fields[2], None, None)

    # We need to emit all of the RIGHT keys that we never saw while streaming through LEFT
    # We will need to deduplicate these in the reducer in case we have multiple mappers
    def mapper_final(self):
        if self.joinType == 'right':
            for key in self.pages:
                if key not in self.seenRight:
                    for value in self.pages[key]:
                        yield key, (None, None, value[0], value[1])
    
    # Need to persist variables
    def reducer_init(self):
        self.joinType = self.options.joinType
    
    # We need to unpack and emit each record
    # We also need to do some work emitting records for the right join
    def reducer(self, key, values):
        emptyRight = True
        for val in values:
            if self.joinType == 'inner' or self.joinType == 'left':
                yield key, val
            elif self.joinType == 'right':
                if val[:2] != [None]*2:
                    emptyRight = False
                    yield key, val
                else: emptyRecord = val
        if emptyRight and self.joinType == 'right':
            yield key, emptyRecord


if __name__ == '__main__':
    join.run()

Overwriting mapSideJoin.py


In [65]:
from mapSideJoin import join

def runJoin(joinType):

    mr_job = join(args=['TopVisitors.txt', '--file', 'attributes.csv', '--joinType', joinType])
    output = []

    with mr_job.make_runner() as runner: 
        # Run MRJob
        runner.run()

        # Write stream_output to file
        for line in runner.stream_output():
            output.append(mr_job.parse_output_line(line))
    
    return output
            
outInner = runJoin('inner')
outLeft = runJoin('left')
outRight = runJoin('right')

In [66]:
print "Rows resulting from join type:\n"
for joinType in ['inner', 'left', 'right']:
    if joinType == 'inner': out = outInner
    elif joinType == 'left': out = outLeft
    elif joinType == 'right': out = outRight
    
    print "{:7s}{:>4,d}".format(joinType, len(out))


Rows resulting from join type:

inner   285
left    285
right   294


## HW5.3: Do some EDA on this data set using MRJob:
- Longest 5-gram (number of characters)
- Top 10 most frequent words (count), i.e., unigrams
- Most/least densely appearing words (count/pages_count) sorted in decreasing order of relative frequency
- Distribution of 5-gram sizes (counts) sorted in decreasing order of relative frequency

In [78]:
import csv

docs = {}
docs['A'] = ['X']*20
docs['A'].extend(['Y']*30)
docs['A'].extend(['Z']*5)
docs['B'] = ['X']*100
docs['B'].extend(['Y']*20)
docs['C'] = ['M']*5
docs['C'].extend(['N']*20)
docs['C'].extend(['Z']*5)


# Create the file for unit testing
with open('unitTest.txt', 'w') as myfile:
    outWriter = csv.writer(myfile)
    for doc in docs:
        row = [doc]
        row.extend(docs[doc])
        outWriter.writerow(row)

In [43]:
%%writefile MRJob5_4.py
from mrjob.job import MRJob
from mrjob.step import MRStep
 
class job(MRJob):
    
    # Specify some custom options so we only have to write one MRJob class for each part
    def configure_options(self):
        super(job, self).configure_options()
        self.add_passthrough_option('--part', default='1')
    
    """
    Find the longest 5-gram
    - In this case, in each mapper, we only need to store the length of the longest 5-gram we have seen
    - After the mapper has run, we emit the longest 5-gram from this mapper
    - All results will be sent to the same reducer (key = None)
    """
    
    def mapper_longest5gram_init(self):
        self.maxLength = 0
    
    def mapper_longest5gram(self, _, line):
        fields = line.strip().split('\t')
        if len(fields[0]) > self.maxLength: 
            self.maxLength = len(fields[0])
            
    def mapper_longest5gram_final(self):
        yield None, self.maxLength
    
    def reducer_longest5gram(self, key, values):
        yield None, max(values)
    
    """
    Top 10 most frequent words
    """

    def mapper_top10words_init(self):
        self.
    
    def mapper_top10words(self, _, line):
        fields = line.strip().split('\t')
        words = fields[0].lower().split()
        count, pages_count, books_count = fields[1], fields[2], fields[3]
        for word in words:
            yield word, (count, pages_count)
    
#     def mapper_top10words
        
    
    # Multi-step pipeline definition
    def steps(self):
        self.part = self.options.part
        if self.part == '1':
            return [
                MRStep(mapper_init=self.mapper_longest5gram_init,
                       mapper=self.mapper_longest5gram,
                       mapper_final=self.mapper_longest5gram_final,
                       reducer=self.reducer_longest5gram)
            ]
        elif self.part == '2':
            return [
                MRStep(mapper=self.mapper_top10words)
            ]

if __name__ == '__main__':
    job.run()

Overwriting MRJob5_4.py


In [44]:
from MRJob5_4 import job

def runJob(part):

    mr_job = job(args=['./filtered-5Grams/short-5gram.txt', '--part', str(part)])
    #mr_job = job(args=['./filtered-5Grams/googlebooks-eng-all-5gram-20090715-0-filtered.txt', '--part', str(part)])
    output = []

    with mr_job.make_runner() as runner: 
        # Run MRJob
        runner.run()

        # Write stream_output to file
        for line in runner.stream_output():
            print mr_job.parse_output_line(line)
            
runJob(2)

['a', 'bill', 'for', 'establishing', 'religious']
['a', 'biography', 'of', 'general', 'george']
['a', 'case', 'study', 'in', 'government']
['a', 'case', 'study', 'of', 'female']
['a', 'case', 'study', 'of', 'limited']
['a', "child's", 'christmas', 'in', 'wales']
['a', 'circumstantial', 'narrative', 'of', 'the']
['a', 'city', 'by', 'the', 'sea']
['a', 'collection', 'of', 'fairy', 'tales']
['a', 'collection', 'of', 'forms', 'of']
['a', 'commentary', 'on', 'his', 'apology']
['a', 'comparative', 'study', 'of', 'juvenile']
['a', 'comparison', 'of', 'the', 'properties']
['a', 'conceptual', 'framework', 'and', 'the']
['a', 'conceptual', 'framework', 'for', 'life']
['a', 'concise', 'bibliography', 'of', 'the']
['a', 'continuation', 'of', 'the', 'letters']
['a', 'critical', 'review', 'and', 'a']
['a', 'critique', 'and', 'a', 'guide']
['a', 'defence', 'of', 'the', 'royal']
['a', 'defence', 'of', 'the', 'short']
['a', 'discovery', 'of', 'the', 'real']
['a', 'further', 'look', 'at', 'the']
['a', '

## HW5.4
### 1. Build stripes of word co-occurrence for the top 10,000
### 2. Using two (symmetric) comparison methods of your choice, pairwise compare all stripes